# Trabalho Prático 1

 ### Ex.1 : StartUp

Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com algumas restrições:

In [665]:
from ortools.linear_solver import pywraplp
#Variáveis S-salas, D-Dias, T-Tempos, P-Projetos
S = 2
D = 2
T = 2
P = 2
#Cada projeto tem associado o seu id, o lider, o número de reuniões e uma lista dos colaboradores
projetos = {1:[1,2,4],2:[2,2,5]}
# Criar a instância do solver para definir o horário
horario = pywraplp.Solver.CreateSolver('SCIP')

Declaramos a matriz de alocação (x) e colaboradores, como um dicionário de modo a guardar as variáveis
p(Id do projeto),
d(Dia),
t(Tempo) e
s(Sala).

In [666]:
#Criar a matriz x para atribuir projeto, dia, tempo e sala da reunião
x = {}

for p in range(1,P+1):
    x[p] = {}
    for d in range(1,D+1):
        x[p][d] = {}
        for t in range(1,T+1):
            x[p][d][t] = {}
            for s in range(1,S+1):
                    x[p][d][t][s] = horario.BoolVar('x[%i][%i][%i][%i]' % (p,d,t,s))

#Criar a matriz Colab para armazenar os colaboradores associados a cada projeto
colaboradores = {}
for p in range(1,P+1):
    LC = projetos[p][2]
    colaboradores[p] = {}
    for d in range(1,D+1):
        colaboradores[p][d] = {}
        for t in range(1,T+1):
            colaboradores[p][d][t] = {}
            for c in range(1,LC+1):
                colaboradores[p][d][t][c] = {}
                for s in range(1,S+1):
                    colaboradores[p][d][t][c][s] = horario.BoolVar('x[%i][%i][%i][%i][%i]' % (p,d,t,c,s))
#Funções para o auxilio da resolução
def X(p,d,t,s):
    return x[p][d][t][s]
def Colab(p,d,t,c,s):
    return colaboradores[p][d][t][c][s]

#Matriz completa

Restrições:

O líder do projeto participa em todas as reuniões do seu projeto. Sendo que $nr$ é o número de reuniões associado a cada projeto.
$$\forall_{p} .\sum_{d<=D,t<=T,s<=S}  X_{p,d,t,s} <= nr$$
$$\forall_{p} .\sum_{d<=D,t<=T,s<=S}  Colab_{p,d,t,lider,s} <= nr$$

In [667]:
for p in range(1,P+1):
    lider = projetos[p][0]
    nr = projetos[p][1]
    #Verificar que cada projeto se tem o número de reuniões correto
    horario.Add(sum([X(p,d,t,s) for d in range(1,D+1) for t in range(1,T+1) for s in range(1,S+1)]) == nr)
    #Verificar se o lider de cada projeto está presente em todas as reuniões
    horario.Add(sum([Colab(p,d,t,lider,s) for d in range(1,D+1) for t in range(1,T+1) for s in range(1,S+1)]) == nr)

A sala não pode ser usada por mais de uma reunião ao mesmo tempo.
$$\forall_{s<=S} . \forall_{d<=D} . \forall_{t<=T} . \sum_{p = 0}^{P}  x_{p,d,t,s} <= 1$$


In [668]:
#Cada sala não pode ter mais que uma reunião ao mesmo tempo
for d in range(1,D+1):
    for t in range(1,T+1):
        for s in range(1,S+1):
            horario.Add(sum([X(p,d,t,s) for p in range(1,P+1)]) <=1)

Cada coloborador não pode estar em mais de uma sala ao mesmo tempo.
$$\forall_{p<=P} . \forall_{d<=D} . \forall_{t<=T} . \sum_{s = 0}^{S}  X_{p,d,t,s} <= nr$$
$$\forall_{p<=P} . \forall_{d<=D} . \forall_{t<=T} . \forall_{c<=C} . \sum_{s = 0}^{S}  Colab_{p,d,t,c,s} <= 1$$


In [669]:
for p in range(1,P+1):
    C = projetos[p][2]
    for c in range(1,C+1):
        for d in range(1,D+1):
            for t in range(1,T+1):
                #Um projeto não pode ter reuniões em mais de uma sala ao mesmo tempo
                horario.Add(sum([X(p,d,t,s) for s in range(1,S+1)]) <= nr)
                #Um colaborador não pode estar em mais de uma sala ao mesmo tempo
                horario.Add(sum([Colab(p,d,t,c,s) for s in range(1,S+1)]) <= 1)
    

Cada reunião tem de ter presente, no mínimo, 50% dos colaboradores do prejeto em questão.
$$\forall_{p<=P} . \forall_{d<=D} . \forall_{t<=T} . \forall_{s<=S} . \sum_{c = 0}^{C}  Colab_{p,d,t,c,s} >= X(p,d,t,s) $$

In [670]:
#Tem de estar no mínimo 50% dos colaboradores em cada reunião
for p in range(1,P+1):
    C = projetos[p][2]
    for d in range(1,D+1):
        for t in range(1,T+1):
            for s in range(1,S+1):
                horario.Add(X(p,d,t,s) <= sum([Colab(p,d,t,c,s) for c in range(1,C+1)])/C)
                

In [672]:
#Invocar o solver para o horario
status = horario.Solve()
#Se possivel imprimir o horário
if status == pywraplp.Solver.OPTIMAL:
    for p in range(1,P+1):
        C = projetos[p][2]
        print('Projeto Nrº:',p)
        for s in range(1,S+1):
            for d in range(1,D+1):
                for t in range(1,T+1):
                    for c in range(1,C+1):
                        if round(Colab(p,d,t,c,s).solution_value()) == 1 and round(X(p,d,t,s).solution_value()) == 1:
                            print('Colaborador:',c,'Sala:',s,'Dia:',d,'Tempo:',t)
                                                
else:
    print("oops...")

Projeto Nrº: 1
Colaborador: 1 Sala: 1 Dia: 1 Tempo: 1
Colaborador: 2 Sala: 1 Dia: 1 Tempo: 1
Colaborador: 3 Sala: 1 Dia: 1 Tempo: 1
Colaborador: 4 Sala: 1 Dia: 1 Tempo: 1
Colaborador: 1 Sala: 1 Dia: 1 Tempo: 2
Colaborador: 2 Sala: 1 Dia: 1 Tempo: 2
Colaborador: 3 Sala: 1 Dia: 1 Tempo: 2
Colaborador: 4 Sala: 1 Dia: 1 Tempo: 2
Projeto Nrº: 2
Colaborador: 1 Sala: 1 Dia: 2 Tempo: 1
Colaborador: 2 Sala: 1 Dia: 2 Tempo: 1
Colaborador: 3 Sala: 1 Dia: 2 Tempo: 1
Colaborador: 4 Sala: 1 Dia: 2 Tempo: 1
Colaborador: 5 Sala: 1 Dia: 2 Tempo: 1
Colaborador: 1 Sala: 1 Dia: 2 Tempo: 2
Colaborador: 2 Sala: 1 Dia: 2 Tempo: 2
Colaborador: 3 Sala: 1 Dia: 2 Tempo: 2
Colaborador: 4 Sala: 1 Dia: 2 Tempo: 2
Colaborador: 5 Sala: 1 Dia: 2 Tempo: 2
